In [1]:
import pandas as pd
from angle_emb import AnglE
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/data-with-media/steam.csv")
df.sample(3)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
6073,382110,Virtual Desktop,2016-03-31,1,Guy Godin,Guy Godin,windows,0,VR Support;Steam Workshop,Utilities,Utilities;VR;VR Only,0,723,316,16,16,100000-200000,10.99
1389,212700,Party of Sin,2012-12-13,1,Crankshaft Games,Crankshaft Games,windows,0,Single-player;Co-op;Shared/Split Screen;Steam ...,Action;Adventure;Indie,Action;Indie;Adventure,33,98,89,290,357,50000-100000,4.79
283,11360,Chains,2009-07-01,1,2DEngine.com,Meridian4,windows;mac,0,Single-player,Indie,Indie;Puzzle;Physics,0,433,148,36,36,100000-200000,3.99


In [3]:
media_df = pd.read_csv("data/data-with-media/steam_media_data.csv")
media_df.sample(3)

,steam_appid,header_image,screenshots,background,movies
8923,464110,https://steamcdn-a.akamaihd.net/steam/apps/464...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/464...,"[{'id': 256664827, 'name': 'Trailer', 'thumbna..."
3337,303840,https://steamcdn-a.akamaihd.net/steam/apps/303...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/303...,"[{'id': 2032611, 'name': 'Gameplay Trailer', '..."
21219,821720,https://steamcdn-a.akamaihd.net/steam/apps/821...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/821...,"[{'id': 256711072, 'name': '20.000 leagues und..."


In [4]:
games_df = pd.read_csv("data/data-with-media/steam.csv")
media_df = pd.read_csv("data/data-with-media/steam_media_data.csv")
description_df = pd.read_csv("data/data-with-media/steam_description_data.csv")


In [5]:
df = games_df.merge(media_df, left_on="appid" , right_on="steam_appid").merge(description_df, on="steam_appid")
game_texts = []
for _, row in df.iterrows():
    game_text = f'''Game name : {row["name"]}
Developer : {row["developer"]}
Genres : {row["genres"]}
Summary : {row["short_description"]}'''
    game_texts.append(game_text)
df["text"] = game_texts
df["minimum_owners"] = df ["owners"].apply(lambda v : int(v.split("-")[0]))
df.sample(3).T

,309,24658,18376
appid,12460,939330,745250
name,BC Kings,Tailor Tales,YAPP: Yet Another Puzzle Platformer
release_date,2009-06-16,2019-01-09,2017-12-01
english,1,1,1
developer,Mascot Entertainment,Celianna,Yet Another Game Studio
publisher,Strategy First,Pixanna,Yet Another Game Studio
platforms,windows,windows,windows
required_age,0,0,0
categories,Single-player;Multi-player,Single-player,Single-player;Steam Achievements
genres,Strategy,Casual;Free to Play;Simulation,Indie


In [6]:
df["minimum_owners"] = df ["owners"].apply(lambda v : int(v.split("-")[0]))
df = df.sort_values("minimum_owners",ascending = False)

In [7]:
from IPython.display import display , HTML
def display_game(row):
    html =''
    html +=  f'<h3>{row["name"]}</h3>'
    html +=  f'<strong>{row["genres"]}</strong>'
    html +=  f'<p>{row["short_description"]}</p>'
    html +=  f'<img src ="{row["header_image"]}">'
    display(HTML(html))
    
for _, row in df.sample(1, random_state=0).iterrows():
    display_game(row)

In [8]:
import numpy as np
top_df = df.sort_values("minimum_owners",ascending = False).head(2000)
embeddings = []
batches = np.array_split(top_df["text"], len(top_df) // 50)
for idx, chunk_text in enumerate(batches):
    if idx % 5 == 0:
        print(f"{idx} / {len(batches)}")
    embeddings +=  list(angle.encode(list(chunk_text), to_numpy=True))
top_df["embedding"] = embeddings

C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


0 / 40
5 / 40
10 / 40
15 / 40
20 / 40
25 / 40
30 / 40
35 / 40


In [9]:
from sklearn.metrics.pairwise import cosine_distances
game_name = 'Counter-Strike: Global Offensive'
game_row = top_df[top_df["name"] == game_name].iloc[0]
distances = cosine_distances(np.array([game_row.embedding]), np.array(top_df.embedding.tolist()))[0]
sorted_indices = distances.argsort()
for idx in sorted_indices[:20]:
    similar_game = top_df.iloc[idx]
    display_game(similar_game)

In [10]:
game_texts = []
for _, row in df.iterrows():
    game_text = f'''Game name : {row["name"]}
Developer : {row["developer"]}
Genres : {row["genres"]}
Summary : {row["short_description"]}'''
    game_texts.append(game_text)
df["text"] = game_texts

In [11]:
top_df.to_parquet("data/game_database.parquet")

In [12]:
import textdistance

In [13]:
textdistance.levenshtein("path of ele","path of exile")

2

In [15]:
top_df["name"].apply(lambda v : textdistance.levenshtein(v, "path of exale")).sort_values()

1772     3
2357     6
4234     7
1759     7
7138     8
        ..
7027    68
704     69
2175    72
4934    88
4994    96
Name: name, Length: 2000, dtype: int64

In [16]:
top_df.loc[1772]

appid                                                              238960
name                                                        Path of Exile
release_date                                                   2013-10-23
english                                                                 1
developer                                             Grinding Gear Games
publisher                                             Grinding Gear Games
platforms                                                         windows
required_age                                                            0
categories              Single-player;Multi-player;Online Multi-Player...
genres                  Action;Adventure;Free to Play;Indie;Massively ...
steamspy_tags                      Free to Play;Action RPG;Hack and Slash
achievements                                                          120
positive_ratings                                                    71593
negative_ratings                      

In [ ]:
print(top_df["name"].head(40).tolist())

In [ ]:
df["embedding"] = list(angle.encode(df["text"].tolist(), to_numpy=True))

In [ ]:
list(angle.encode(df["short_description"].tolist()[:100], to_numpy=True))

In [ ]:
distances[distances.argsort()]

In [ ]:
#pip install -U angle-emb

In [ ]:
#pip install tf-keras

In [ ]:
vec = angle.encode('hello world', to_numpy = True)
print(vec)
vecs=angle.encode(['hello world1','hello world2'], to_numpy=True)
print(vecs)

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(angle.encode(['beef stakes is delicious ',
                              'the capital of Morocco is Rabat',
                              'I love meat']))[0]

In [ ]:
#Minute 1:12:20